# Lorenz system: manifold plots

In [ ]:
%matplotlib osx

import sys
import numpy as np
import matplotlib
import torch
import logging
import pickle
from tqdm import tqdm
from scipy.spatial import cKDTree

from matplotlib import pyplot as plt, cm
from matplotlib.collections import PolyCollection
from matplotlib.tri import Triangulation
from matplotlib.colors import LightSource
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

sys.path.append("../../")
from experiments.datasets import LorenzSimulator
from experiments.architectures.vector_transforms import create_vector_transform
from manifold_flow.flows import ManifoldFlow
import plot_settings as ps

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-30.30s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.DEBUG,
)
for key in logging.Logger.manager.loggerDict:
    if "experiments" not in key and "manifold_flow" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [ ]:
# ps.setup()

## Options

In [3]:
mode = "sample"

## Set up simulator

In [4]:
sim = LorenzSimulator(random_trajectories=10)


## Load flows

In [5]:
def load_model(filename):
    outer_transform = create_vector_transform(
        3,
        5,
        #linear_transform_type="permutation",
        linear_transform_type="lu",
        base_transform_type="rq-coupling",
        context_features=None,
        dropout_probability=0.,
        tail_bound=10., # 3.,
        num_bins=5,
    )
    inner_transform = create_vector_transform(
        2,
        5,
        #linear_transform_type="permutation",
        linear_transform_type="lu",
        base_transform_type="rq-coupling",
        context_features=sim.parameter_dim(),
        dropout_probability=0.,
        tail_bound=10., # 3.,
        num_bins=5,
    )

    model = ManifoldFlow(
        data_dim=3,
        latent_dim=2,
        outer_transform=outer_transform,
        inner_transform=inner_transform,
        apply_context_to_outer=False,
    )
    
    model.load_state_dict(
        torch.load("../data/models/{}.pt".format(filename), map_location=torch.device("cpu"))
    )
    _ = model.eval()
    
    return model


mf = load_model("mf_2_lorenz_may")
# mf = load_model("mf_2_lorenz_april2")


10:03 manifold_flow.transforms.proje DEBUG   Set up projection from vector with dimension 3 to vector with dimension 2
10:03 manifold_flow.flows.base       DEBUG   Flow has 0.4 M parameters (0.4 M trainable) with an estimated size of 1.7 MB
10:03 manifold_flow.flows.manifold_f DEBUG     Outer transform: 0.2 M parameters
10:03 manifold_flow.flows.manifold_f DEBUG     Inner transform: 0.2 M parameters


## Find points for evaluation: start with latent grid, triangulate, add points randomly where necessary, remove duplicates

In [6]:
def make_grid(boundary, res):
    grid_each = np.linspace(-boundary, boundary, res)
    x, y = np.meshgrid(grid_each, grid_each)
    xy = np.vstack((x.flatten(), y.flatten())).T
    return xy


def decode(zs, batchsize = 10000):
    xs = []
    n_batches = (len(zs) - 1) // batchsize + 1
    for i in range(n_batches):
        print("Batch {} / {}".format(i + 1, n_batches))
        z_batch = zs[i*batchsize:(i+1)*batchsize]
        z_ = torch.tensor(z_batch, dtype=torch.float)
        x_ = mf.decode(u=z_)
        xs += list(x_.detach().numpy().flatten())

    xs = np.array(xs).reshape((-1, 3))
    return xs


def encode(xs, batchsize = 10000):
    zs = []
    n_batches = (len(xs) - 1) // batchsize + 1
    for i in range(n_batches):
        print("Batch {} / {}".format(i + 1, n_batches))
        x_batch = xs[i*batchsize:(i+1)*batchsize]
        x_ = torch.tensor(x_batch, dtype=torch.float)
        z_ = mf.encode(x_)
        zs += list(z_.detach().numpy().flatten())

    zs = np.array(zs).reshape((-1, 2))
    return zs


def triangle_circumference(a, b, c):
    ab = np.linalg.norm(a - b)
    bc = np.linalg.norm(b - c)
    ca = np.linalg.norm(c - a)
    circ = (ab + bc + ca)
    return circ


def triangle_area(a, b, c):
    ab = np.linalg.norm(a - b)
    bc = np.linalg.norm(b - c)
    ca = np.linalg.norm(c - a)
    s = (ab + bc + ca) / 2.
    return (s * (s-ab) * (s-bc) * (s-ca))**0.5
    
    
def uniform_points_in_triangle(a, b, c, n=1):
    p = np.random.uniform(size=n)[:,np.newaxis]
    q = np.random.uniform(size=n)[:,np.newaxis]
    p_ = np.where(p + q > 1., 1. - p, p)
    q_ = np.where(p + q > 1., 1. - q, q)
    points = a[np.newaxis, :] + (b - a)[np.newaxis, :] * p_ + (c - a)[np.newaxis, :] * q_
    
    return points


In [7]:
def iterate(xs, zs, goal_area = 0.01, goal_circ=0.2, insert_max=20, max_distance=0.1, distance_space="mix"):
    print("Beginning iteration")
    n_in = xs.shape[0]
    z_outs = np.copy(zs)
    
    # Step one: inserting more points where distances are large
    print("Triangulating...")
    triangulation = Triangulation(zs[:,0], zs[:,1])

    print("Inserting points in latent space...")
    for t in triangulation.triangles:
        area = triangle_area(xs[t[0]], xs[t[1]], xs[t[2]])
        circ = triangle_circumference(xs[t[0]], xs[t[1]], xs[t[2]])
        try:
            n_additional = int(round(np.clip(
                0.5 * (area / goal_area + circ / goal_circ),
                0., insert_max
            ), 0))
        except:
            n_additional = insert_max // 2  # Seems like a safe choice

        if n_additional > 0:
            z_additional = uniform_points_in_triangle(
                zs[t[0]],
                zs[t[1]],
                zs[t[2]],
                n=n_additional
            )
            z_outs = np.concatenate((z_outs, z_additional), axis=0)
            
    print("Decoding...")
    x_outs = decode(z_outs)
    n_mid = len(x_outs)

    # Step two: removing close neighbors in (x,z) space
    print(f"Removing close neighbors in {distance_space} space...")
    if distance_space == "mix":
        tree = cKDTree(np.concatenate([x_mf, z_mf], axis=1))
    elif distance_space == "x":
        tree = cKDTree(x_mf)
    elif distance_space == "z":
        tree = cKDTree(z_mf)
    else:
        raise ValueError(distance_space)
    neighbors = tree.query_pairs(r=max_distance)
    duplicates = [n[1] for n in neighbors]

    x_outs = np.array([x for i, x in enumerate(x_outs) if i not in duplicates])
    z_outs = np.array([z for i, z in enumerate(z_outs) if i not in duplicates])
    n_out = len(x_outs)
    
    print(f"Done! {n_in} -> {n_mid} -> {n_out}")

    return x_outs, z_outs

In [8]:
if mode == "latent":
    z_mf = make_grid(2.9, 100)
    x_mf = decode(z_mf)
    
    for _ in range(2):
        x_mf, z_mf = iterate(x_mf, z_mf)
        
elif mode == "mix":
    x_mf = sim.sample(1000)
    z_mf = encode(x_mf)
    
    for _ in range(2):
        x_mf, z_mf = iterate(x_mf, z_mf)
        
elif mode == "sample":
    x_mf = sim.sample(25000)
    z_mf = encode(x_mf)
    
    for _ in range(1):
        x_mf, z_mf = iterate(x_mf, z_mf, insert_max=0, distance_space="x")


10:03 experiments.datasets.lorenz    INFO    Solving Lorenz system with sigma = 10.0, beta = 2.6666666666666665, rho = 28.0, initial conditions x0 = [1.19529968 1.06969531 0.88328881], saving 1000000 time steps from 50.0 to 1000.0
10:03 experiments.datasets.lorenz    DEBUG   Done
10:03 experiments.datasets.lorenz    INFO    Solving Lorenz system with sigma = 10.0, beta = 2.6666666666666665, rho = 28.0, initial conditions x0 = [0.9799591  0.9581592  0.87777382], saving 1000000 time steps from 50.0 to 1000.0
10:03 experiments.datasets.lorenz    DEBUG   Done
10:03 experiments.datasets.lorenz    INFO    Solving Lorenz system with sigma = 10.0, beta = 2.6666666666666665, rho = 28.0, initial conditions x0 = [1.013473   0.89268983 1.08024746], saving 1000000 time steps from 50.0 to 1000.0
10:04 experiments.datasets.lorenz    DEBUG   Done
10:04 experiments.datasets.lorenz    INFO    Solving Lorenz system with sigma = 10.0, beta = 2.6666666666666665, rho = 28.0, initial conditions x0 = [1.05454

Batch 1 / 3
Batch 2 / 3
Batch 3 / 3
Beginning iteration
Triangulating...
Inserting points in latent space...
Decoding...
Batch 1 / 3
Batch 2 / 3
Batch 3 / 3
Removing close neighbors in x space...
Done! 25000 -> 25000 -> 1036


## Evaluate log likelihood

In [9]:
batchsize = 600
n_batches = (len(z_mf) - 1) // batchsize + 1

logps = []

for i in range(n_batches):
    print("Batch {} / {}".format(i + 1, n_batches))
    x_batch = x_mf[i*batchsize:(i+1)*batchsize]
    x_batch_ = torch.tensor(x_batch, dtype=torch.float)
    print(x_batch_.size())
    _, logp_, _ = mf(x_batch_, mode="mf")
    logps.append(logp_.detach().numpy().flatten())

logp_mf = np.hstack(logps)
logp_mf.shape


Batch 1 / 2
torch.Size([600, 3])


RuntimeError: Expected tensor to have size 600 at dimension 0, but got size 1 for argument #2 'batch2' (while checking arguments for bmm)

In [ ]:
x_traj = sim.trajectory(0)
z_traj = encode(x_traj, batchsize=10000)
x_traj.shape, z_traj.shape


## Save / load results

In [26]:
pickle.dump((x_mf, z_mf, logp_mf, x_traj, z_traj), open( "../data/results/lorenz.pickle", "wb"))

# if latent_seeds:
#     pickle.dump((x_mf, z_mf, logp_mf, x_traj, z_traj), open( "../data/results/lorenz2.pickle", "wb"))
# else:
#     pickle.dump((x_mf, z_mf, logp_mf, x_traj, z_traj), open( "../data/results/lorenz3.pickle", "wb"))


In [5]:
# x_mf, z_mf, logp_mf, x_traj, z_traj = pickle.load(open( "../data/results/lorenz.pickle", "rb"))


## Scatter plots

In [6]:
logp_cutoff = -15.

cmin, cmax = -7., -2.
x_boundary = 3.

def cmap(vals):
    return ps.CMAP(np.clip((vals - cmin) / (cmax - cmin), 0., 1.))

if logp_cutoff is None:
    x_ = x_mf
    colors_ = cmap(logp_mf)
else:
    x_ = x_mf[logp_mf > logp_cutoff]
    colors_ = cmap(logp_mf[logp_mf > logp_cutoff])


In [8]:
# Strongly based on https://discourse.matplotlib.org/t/trisurf-plots-with-independent-color-data/19033/2

plt.close("all")
fig = plt.figure(figsize=(6,6))
ax = fig.gca(projection='3d')
    
ax.scatter(x_[:, 0], x_[:, 1], x_[:,2], c=colors_, s=2)

ax.set_xlim3d(-x_boundary, x_boundary)
ax.set_ylim3d(-x_boundary, x_boundary)
ax.set_zlim3d(-x_boundary, x_boundary)

ax.set_xlabel("$x_0$")
ax.set_ylabel("$x_1$")
ax.set_zlabel("$x_2$")

# ax.set_xticklabels([""]*3)
# ax.set_yticklabels([""]*3)
# ax.set_zticklabels([""]*3)

ax.view_init(elev=20, azim=-70)  # Good view
plt.tight_layout()
plt.savefig("../figures/lorenz_likelihood_3d_scatter_front.png")

ax.view_init(elev=20, azim=80)  # Another good view
plt.tight_layout()
plt.savefig("../figures/lorenz_likelihood_3d_scatter_half.png")

ax.view_init(elev=20, azim=-70+180)  # Back view
plt.tight_layout()
plt.savefig("../figures/lorenz_likelihood_3d_scatter_back.png")

ax.view_init(elev=20, azim=60)  # Great side view
plt.tight_layout()
plt.savefig("../figures/lorenz_likelihood_3d_scatter_side.png")

ax.view_init(elev=90, azim=-70)  # Top-down view of x-y plane
plt.tight_layout()
plt.savefig("../figures/lorenz_likelihood_3d_scatter_top.png")

for azim in [0, 45, 90, 135, 180, 225, 270, 295]:  # Rotate around z axis
    ax.view_init(elev=0, azim=azim)
    plt.tight_layout()
    plt.savefig(f"../figures/lorenz_likelihood_3d_scatter_{azim}.png")



## Triangulation

In [28]:
x_tri = []
logp_tri = []
z_tri = []
minlogp_tri = []
circ_tri = []

x_ = x_mf
z_ = z_mf
logp_ = logp_mf

tri = Triangulation(z_[:,0], z_[:,1])

for t in tri.triangles:
    z_tri.append(
        [
            [z_[t[0], 0], z_[t[0], 1]],
            [z_[t[1], 0], z_[t[1], 1]],
            [z_[t[2], 0], z_[t[2], 1]]
        ]
    )
    x_tri.append(
        [
            [x_[t[0], 0], x_[t[0], 1], x_[t[0], 2]],
            [x_[t[1], 0], x_[t[1], 1], x_[t[1], 2]],
            [x_[t[2], 0], x_[t[2], 1], x_[t[2], 2]]
        ]
    )
    logp_tri.append((logp_[t[0]] + logp_[t[1]] + logp_[t[2]]) / 3.)
    minlogp_tri.append(min(logp_[t[0]], logp_[t[1]], logp_[t[2]]))
    circ_tri.append(triangle_circumference(x_[t[0]], x_[t[1]], x_[t[2]]))

z_tri = np.array(z_tri)
x_tri = np.array(x_tri)
logp_tri = np.array(logp_tri)
minlogp_tri = np.array(minlogp_tri)
circ_tri = np.array(circ_tri)

logp_tri.shape, x_tri.shape, z_tri.shape, circ_tri.shape


((66325,), (66325, 3, 3), (66325, 3, 2), (66325,))

## Preprocessing for plotting

In [41]:
logp_cutoff = -15.
circ_cutoff = 1

cmin, cmax = -7., -2.
x_boundary = 3.

def cmap(vals):
    return ps.CMAP(np.clip((vals - cmin) / (cmax - cmin), 0., 1.))

if logp_cutoff is None:
    x_tri_ = x_tri
    colors_tri_ = cmap(logp_tri)
    z_tri_ = z_tri
    circ_tri_ = circ_tri
else:
    x_tri_ = x_tri[minlogp_tri > logp_cutoff]
    colors_tri_ = cmap(logp_tri[minlogp_tri > logp_cutoff])
    z_tri_ = z_tri[minlogp_tri > logp_cutoff]
    circ_tri_ = circ_tri[minlogp_tri > logp_cutoff]

if circ_cutoff is not None:
    x_tri_ = x_tri_[circ_tri_ < circ_cutoff]
    colors_tri_ = colors_tri_[circ_tri_ < circ_cutoff]
    z_tri_ = z_tri_[circ_tri_ < circ_cutoff]


## 2D plot

In [42]:
fig = plt.figure(figsize=(6,6))
ax = plt.gca()

coll = PolyCollection(z_tri_, facecolors=colors_tri_, edgecolors=colors_tri_, lw=0.5, rasterized=True)
ax.add_collection(coll)

# ax.plot(z_traj[:, 0], z_traj[:, 1], c="black", lw=0.5)

plt.xlabel("$u_1$")
plt.ylabel("$u_2$")

plt.xlim(-3, 3)
plt.ylim(-3, 3)

plt.tight_layout()
plt.savefig("../figures/lorenz_latent_likelihood.pdf")


In [43]:
def xcmap(vals):
    return ps.CMAP(np.clip((vals - (-3)) / (3 - (-3)), 0., 1.))

plt.close("all")
fig = plt.figure(figsize=(12,4))

for i in range(3):
    ax = plt.subplot(1,3,i+1)
    colors = xcmap(np.mean(x_tri_[:,:,i], axis=1))
    coll = PolyCollection(z_tri_, facecolors=colors, edgecolors=colors, lw=0.5, rasterized=True)
    ax.add_collection(coll)
    plt.xlabel("$u_1$")
    plt.ylabel("$u_2$")
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.title(f"$x_{i+1}$", fontsize=11)

plt.tight_layout()
plt.savefig("../figures/lorenz_latent_x.pdf")


## 3D plot

In [45]:
# Strongly based on https://discourse.matplotlib.org/t/trisurf-plots-with-independent-color-data/19033/2

plt.close("all")
fig = plt.figure(figsize=(6,6))
ax = fig.gca(projection='3d')
    
coll = Poly3DCollection(x_tri_, facecolors=colors_tri_, edgecolors=colors_tri_, lw=0.5)
ax.add_collection(coll)

ax.set_xlim3d(-x_boundary, x_boundary)
ax.set_ylim3d(-x_boundary, x_boundary)
ax.set_zlim3d(-x_boundary, x_boundary)

ax.set_xlabel("$x_0$")
ax.set_ylabel("$x_1$")
ax.set_zlabel("$x_2$")

# ax.set_xticklabels([""]*3)
# ax.set_yticklabels([""]*3)
# ax.set_zticklabels([""]*3)

ax.view_init(elev=20, azim=-70)  # Good view
plt.tight_layout()
plt.savefig("../figures/lorenz_likelihood_3d_front.png")

ax.view_init(elev=20, azim=80)  # Another good view
plt.tight_layout()
plt.savefig("../figures/lorenz_likelihood_3d_half.png")

ax.view_init(elev=20, azim=-70+180)  # Back view
plt.tight_layout()
plt.savefig("../figures/lorenz_likelihood_3d_back.png")

ax.view_init(elev=20, azim=60)  # Great side view
plt.tight_layout()
plt.savefig("../figures/lorenz_likelihood_3d_side.png")

ax.view_init(elev=90, azim=-70)  # Top-down view of x-y plane
plt.tight_layout()
plt.savefig("../figures/lorenz_likelihood_3d_top.png")

for azim in [0, 45, 90, 135, 180, 225, 270, 295]:  # Rotate around z axis
    ax.view_init(elev=0, azim=azim)
    plt.tight_layout()
    plt.savefig(f"../figures/lorenz_likelihood_3d_{azim}.png")
